In [1]:
%pip install maritalk
%pip install pyttsx3
%pip install SpeechRecognition
%pip install pyaudio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from maritalk import MariTalk
import datetime
import pyttsx3
import speech_recognition as sr

with open('key_txt.txt', 'r') as file:
    maritalk_key = file.read().strip()

maritalk_client = MariTalk(key=maritalk_key, model="sabia-2-small")

def listen_and_transcribe():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Ajustando para barulho de ambiente...")
        recognizer.adjust_for_ambient_noise(source)
        print("Escutando...")
        audio = recognizer.listen(source)
    
    try:
        print("Transcrevendo...")
        text = recognizer.recognize_google(audio, language='pt-BR')
        print(f"Transcrição: {text}")
        
        return text

    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")


def text_to_speech(text, lang='pt'):
    engine = pyttsx3.init()
    
    engine.setProperty('rate', 200)  # Speed percent
    engine.setProperty('volume', 1)  # Volume 0-1

    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)

    engine.say(text)
    
    engine.runAndWait()

def update_sensor_data(sensor_data):
    context_message = {
        "role": "system",
        "content": f"Responda o usuário com um bom dia/tarde/noite baseado no horario atual, além disso sempre que o usuário quiser saber a temperatura " + 
        f"cheque as informações a seguir, para saber a hora atual também! Contexto de sensores: {sensor_data}, não faça formatação no texto de resposta."
    }
    return context_message

def receive_message(history, message):
    history.append({"role": "user", "content": message})
    return history

def generate_response(history):
    print_history(history)
    response = maritalk_client.generate(
        history,
        do_sample=True,
        max_tokens=200,
        temperature=0.7,
        top_p=0.95
    )["answer"]
    
    history.append({"role": "assistant", "content": response})
    return response, history

def print_history(history):
    for entry in history:
        if entry["role"] == "system":
            print(f"System: {entry['content']}")
        elif entry["role"] == "user":
            print(f"You: {entry['content']}")
        elif entry["role"] == "assistant":
            print(f"Bot: {entry['content']}")

def chatbot_loop():
    time = datetime.datetime.now().strftime("%H:%M")
    sensor_data = {'temperatura_quarto': 23, 'temperatura_sala': 24, 'horario_agora': time}

    history = [update_sensor_data(sensor_data)]

    while 1:
        print('')
        user_message = listen_and_transcribe()
        if 'ASSISTENTE' in user_message.upper():
            history = receive_message(history, user_message)
            response, history = generate_response(history)
            print(f"Bot: {response}")
            text_to_speech(response)

            time = datetime.datetime.now().strftime("%H:%M")
            sensor_data = {'temperatura_quarto': 24, 'temperatura_sala': 26, 'horario_agora': time}
            context_message = update_sensor_data(sensor_data)

            history = [context_message] + history[1:]
        elif 'PARAR CONVERSA' in user_message.upper():
            break
        else:
            continue

In [7]:
chatbot_loop()


Ajustando para barulho de ambiente...
Escutando...
Transcrevendo...
Transcrição: eu criei aqui tipo um chatbot é um loop aí ele fica escutando o que eu tô falando aqui no microfone e aí ele transcreve isso e envia para o modelo porém ele só manda para o modelo se ele escutar essa palavra aqui no meio do texto Então vou deixar vou ficar parado vou parar de falar um pouco para ele parar de transcrever

Ajustando para barulho de ambiente...
Escutando...
Transcrevendo...
Transcrição: assistente Quantos graus está agora no quarto Além disso me dê uma receita de bolo de chocolate enquanto eu estou indo para lá
System: Responda o usuário com um bom dia/tarde/noite baseado no horario atual, além disso sempre que o usuário quiser saber a temperatura cheque as informações a seguir, para saber a hora atual também! Contexto de sensores: {'temperatura_quarto': 23, 'temperatura_sala': 24, 'horario_agora': '19:49'}, não faça formatação no texto de resposta.
You: assistente Quantos graus está agora n